In [13]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

tfd = tfp.distributions



In [43]:
c = np.log(np.expm1(1.))
c

0.5413248546129181

In [42]:
w = tf.Variable(np.random.randn(12,4,1))
x = tf.Variable(np.random.randn(12,1))
einsum_matrix_mult = 'Bup,Bp->Bu'.format('Bup')
#x = tf.einsum(einsum_matrix_mult, w, x)
tf.einsum(einsum_matrix_mult, w, x)

<tf.Tensor: id=927, shape=(12, 4), dtype=float64, numpy=
array([[ 0.36703832, -1.70379857, -1.10698914,  1.16562992],
       [ 0.20900578, -0.28255743,  0.11396599,  0.43205455],
       [ 0.05073713,  0.49162993,  0.15200601, -0.08125023],
       [ 0.22375872, -0.09579545, -0.0381696 ,  0.29882579],
       [ 0.17290477,  0.12262933, -0.19743183, -0.08979189],
       [-0.66357466, -0.88756377,  0.34494701, -0.0289138 ],
       [-1.50206008,  0.01201194,  0.25424973, -2.25047018],
       [ 0.02100421,  0.01596982,  0.06085757, -0.02537529],
       [ 0.2533604 ,  0.02516034, -0.34893812,  0.13302887],
       [ 0.04456626,  0.42969639, -0.965884  ,  0.27591499],
       [-3.48460223, -1.29490439,  0.74429302,  1.13171027],
       [-0.73885167,  0.69469092, -0.25126293, -0.60388343]])>

In [10]:
n=4
c=1
posterior = tf.keras.Sequential([
            tfp.layers.VariableLayer([20,2 * n]),
            tfp.layers.DistributionLambda(lambda t: tfd.Independent(
                tfd.Normal(loc=t[..., :n],
                           scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
                reinterpreted_batch_ndims=1)),
        ])

In [20]:
posterior(3)

<tfp.distributions.Independent 'Independentsequential_1/distribution_lambda_1/Normal/' batch_shape=[20] event_shape=[4] dtype=float32>

In [19]:
tf.gather(posterior(3), np.array([2]))

<tfp.distributions.Independent 'Independentsequential_1/distribution_lambda_1/Normal/' batch_shape=[20] event_shape=[4] dtype=float32>

In [30]:
tfd.kl_divergence(tfd.Normal([0.,0.],tf.nn.softplus([1.,1.])), tfd.Normal(3.,tf.nn.softplus(2.)))

<tf.Tensor: id=790, shape=(2,), dtype=float32, numpy=array([1.167518, 1.167518], dtype=float32)>

In [28]:
def compute_kl( mu1, sigma1, mu2, sigma2):
    sigma1_plus = tf.nn.softplus(sigma1)
    sigma2_plus = tf.nn.softplus(sigma2)
    kl = (
        tf.math.log(sigma2_plus/sigma1_plus)
        + (sigma1_plus**2 + (mu1-mu2)**2)/(2*sigma2_plus**2)
        - 0.5)

    return kl

In [34]:
compute_kl(np.array([0.,0.],dtype=np.float32),np.array([1.,1.],dtype=np.float32),3.,2.)

<tf.Tensor: id=829, shape=(2,), dtype=float32, numpy=array([1.167518, 1.167518], dtype=float32)>